## Graph Execution in Tensorflow

We are using `eager execution` by default in tensorflow 2.0

Tensorflow can also be run in `Graph` mode where operations are run entirely run in `Tensorflow Graphs`. 

Graphs are data `structures` which contains the `tf.Operation` objects in nodes and 'tf.Tensor' objects in edges. Since they are not python data structures, they can be saved, restored and run without python being present

**What is `tf.function`?**

`tf.function` is the bridge between normal eager execution and graph execution. How does it do that --> It uses something called `AutoGraph`

**What is AutoGraph?**

In [24]:
import tensorflow as tf
import numpy as np

### Converting python functions to tensorflow graphs

The normal python code can be converted to a `Function` using the `tf.function` decorator offered by tensorflow

In [228]:
def add(x,y):
    print("adding")
    result = tf.add(x,y)
    return result

In [229]:
a = tf.random.uniform((10,10))

In [231]:
b = tf.random.uniform((10,10))

`add` is a python function

In [238]:
add

<function __main__.add(x, y)>

In [241]:
@tf.function
def add(x,y):
    print("adding")
    result = tf.add(x,y)
    return result

In [242]:
add

Notice that the type of `add` has changed to `Function`

In [243]:
for x in range(5):
    _ = add(a,b)

adding


Notice that even the though we called the graph function 5 times, it only printed `adding` once. This is due to `Tracing`


**Tracing**<br>
When the graph is defined the operation inside is recored and embedded into the graph. Tracing ignores operations like python print function as they are not very important. Unless you change the input data type, the `Tracing` is done only once and while calling the function the graph simply runs the saved operations as opposed normal python function where it execute the whole function line by line

Note that a 'Function' can be excuted in the eager mode by chancging the default mode by doing the following:

In [245]:
tf.config.run_functions_eagerly(False)

### Speed Up

In [212]:
@tf.function
def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

In [213]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

### Running in Graph Mode

In [214]:
%timeit -n 1000 power(x, 100)

291 µs ± 44.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Running in `Eager` mode

In [215]:
tf.config.run_functions_eagerly(True)

In [216]:
%timeit -n 1000 power(x, 100)

1.62 ms ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The graph execution is about `5x` faster in this case. Note that this is not a generalized benchmark. The actual speed-up depends on the functions you use. 


The graph creating process takes some time and that adds up to the execution time. This is a one time process and the loss in time is compensated by performance boost for repeated execution of the function. 

This can also end up making the first few step of training loop more faster than eager execution, but eventually the whole training is much faster.

### things to keep in mind while using `tf.function` using 

1. Include as much operations you can under the `tf.function` hood
2. Pass tensorflow datatypes as inputs such as `tf.Tensor` or `tf.Variable`

#### As this is an important topic, more example providing things to do or avoid needs to be included

### Does Keras training happen in graphs?

Yes. By default keras models are run in graph mode. They do it by wrapping their training steps under the `tf.function` hood

Have a look the following code to check how they exactly implement it

`https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/keras/engine/training.py#L863`

Note: Tensorflow offers a function to convert any function to `tf.Function`

`tf.python.eager.def_function.function` and it's exposed as `tf.function` callable publically.

In [2]:
tf.python.eager.def_function.function==tf.function

True